### <span style="color:red"><b>Investigate the Distribution of EV Charger Stations</b></span>
    around major entertainment facilities and other community use sites in city of Melbourne.

Goal: to help understand if there is a demand for future additions around major destinations for people to spend time in the city.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians

Step 0: Prepare functions
- read csv files
- calculate distance between two locations based on geo coordinations

In [2]:
# Read file
'''
input: filename(string)
ouput: df(pandas dataframe)
'''
def read_csv(filename):
    df = pd.read_csv(filename)
    return df


# Calculate distance(in meters) between two locations
'''
input: latitude_1(float),longitude_1(float),latitude_2(float),longitude_2(float)
output: distance(float)
'''
def calculate_distance(latitude_1,longitude_1,latitude_2,longitude_2):
    
    R = 6373.0

    lat1 = radians(latitude_1)
    lon1 = radians(longitude_1)
    lat2 = radians(latitude_2)
    lon2 = radians(longitude_2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c * 1000

    return distance





Step 1: Prepare data for landmarks
- load landmarks and select target locations

    Dataset resources: https://discover.data.vic.gov.au/dataset/ ,    
    "Landmarks and places of interest, including schools, theatres, health services, sports facilities, places of worship, galleries and museums."
    
    Original dataset has 242 entries. Here only locations under the theme of "Community Use", "Leisure/Recreation", "Transport" are selected, since there seems to have obvious demands for EV charger stations around such destinations.
- reset the dataframe with two features: location name, geo coordinates

In [3]:
# load landmarks and select target locations

df_landmarks = read_csv("landmarks.csv")
rows_num = df_landmarks.shape[0]
target_location_list = ['Community Use','Leisure/Recreation','Transport']

for i in range(rows_num):
    if df_landmarks.at[i,'theme'] not in target_location_list:
        df_landmarks.drop(i,axis=0,inplace=True)

df_landmarks.drop(columns=['theme','sub_theme'],inplace=True)


# reset the dataframe with two features: location name, geo coordinates

df_landmarks = df_landmarks.reset_index(drop=True)
print(df_landmarks.head(10))
print(df_landmarks.shape)

                     feature_name                         co_ordinates
0               Port of Melbourne   -37.8137384362671, 144.91753432375
1           Carlton Football Club  -37.7840864379557, 144.961967841559
2           Carlton Gardens North  -37.8017690847403, 144.971997551189
3                    Kings Domain  -37.8255239795833, 144.974107925144
4           Flemington Racecourse  -37.7908082646624, 144.912142987372
5                Treasury Gardens  -37.8143993575938, 144.975952335785
6     Melbourne Magistrates Court  -37.8136147671606, 144.956846193891
7       Flagstaff Railway Station  -37.8122356514626, 144.956318211113
8  Melbourne Cricket Ground (MCG)  -37.8194921618419, 144.983402879078
9          Queen Victoria Gardens  -37.8216381244891, 144.971049530478
(110, 2)


Step 2: For each landmark, get the number of EV charger station around it

- prepare data for EV charger station
- for each landmark, calculate how many EV charger stations are there within the radius of 1 kilometers
- integrate result and write to a csv file

In [4]:
#load EV_charger dataset
df_EV_1 = read_csv('Cleaned_Australian_EV_Charging_Stations.csv')
df_EV_2 = read_csv('EV_Charging_Station_New_Data_2024T1.csv')

print(df_EV_1.shape[0], ' entries in dataset 1 (before 2023)')
print(df_EV_2.shape[0], ' entries in dataset 2 (since 2023)')

391  entries in dataset 1 (before 2023)
79  entries in dataset 2 (since 2023)


In [18]:
# Compare landmark location with data in dataset 1 (df_EV_1)
df_EV_1 = df_EV_1[['Location Name','Latitude','Longitude']]

landmarks_charger_number = {}
landmarks_charger_info = {}
landmarks_location_info = {}

for i in range(df_landmarks.shape[0]):
    geo_co = df_landmarks.at[i,'co_ordinates']
    lati,longi = geo_co.split(', ')

    lat_1 = float(lati)
    lon_1 = float(longi)
    landmark_name = df_landmarks.at[i,'feature_name']

    info_dict = []
    EV_number = 0

    for j in range(df_EV_1.shape[0]):

        EV_name = df_EV_1.at[j,'Location Name']
        lat_2 = df_EV_1.at[j,'Latitude']
        lon_2 = df_EV_1.at[j,'Longitude']
        distance = calculate_distance(lat_1,lon_1,lat_2,lon_2)

        if distance <= 1000:
            EV_number +=1
            info_dict.append(EV_name + ': ' + str(int(distance)) + 'm') 
    
    for n in range(df_EV_2.shape[0]):

        EV_name = df_EV_2.at[n,'Location Name']
        lat_2 = df_EV_2.at[n,'Latitude']
        lon_2 = df_EV_1.at[n,'Longitude']
        distance = calculate_distance(lat_1,lon_1,lat_2,lon_2)

        if distance <= 1000:
            EV_number +=1
            info_dict.append(EV_name + ': ' + str(int(distance)) + 'm') 
        
    landmarks_charger_number[landmark_name] = EV_number
    landmarks_charger_info[landmark_name] = info_dict
    landmarks_location_info[landmark_name] = [lat_1,lon_1]

print(landmarks_charger_number)
print(landmarks_charger_info)
print(landmarks_location_info)

{'Port of Melbourne': 0, 'Carlton Football Club': 0, 'Carlton Gardens North': 6, 'Kings Domain': 9, 'Flemington Racecourse': 0, 'Treasury Gardens': 18, 'Melbourne Magistrates Court': 35, 'Flagstaff Railway Station': 31, 'Melbourne Cricket Ground (MCG)': 2, 'Queen Victoria Gardens': 13, 'Federation Square': 32, 'New Quay': 5, 'New Quay Marina': 7, 'Newmarket Reserve': 0, 'Wonderland Park': 4, 'Rod Laver Arena': 7, 'Melbourne Park': 6, 'Hisense Arena': 2, 'Elisabeth Murdoch Hall': 18, 'Melbourne Recital Centre': 18, 'J.J Holland Park': 2, 'South Kensington Railway Station': 2, 'Royal Park Railway Station': 0, 'Melbourne Central Railway Station': 33, 'Riverside Park': 2, 'Flagstaff Gardens': 26, 'Fawkner Park': 3, 'Fitzroy Gardens': 13, 'Enterprize Park': 31, 'Flinders Street Railway Station': 35, 'Metropolitan Fire Brigade (MFB)': 17, 'Commonwealth Law Courts': 32, 'Macarthur Square': 4, 'City Baths': 23, 'Westgate Park': 6, 'North Melbourne Railway Station': 7, 'Melbourne Childrens Cour

In [19]:
# integrate data to one dataframe and out write to a csv file

data = {'Landmark Name':[],'Latitude':[],'Longitude':[],'Number of EV charger station':[],'Name and distance of each station':[]}

for name,number in landmarks_charger_number.items():
    data['Landmark Name'].append(name)
    
    lat = landmarks_location_info[name][0]
    lon = landmarks_location_info[name][1]
    data['Latitude'].append(lat)
    data['Longitude'].append(lon)

    data['Number of EV charger station'].append(number)
    if number == 0:
        data['Name and distance of each station'].append('N/a')
    else:
        info = ''
        for item in landmarks_charger_info[name]:
            info += item
            info += ', '
        data['Name and distance of each station'].append(info)

df_output = pd.DataFrame.from_dict(data)
df_output.to_csv('EV charger station info around public locations.csv') 


Step 3: Review the result and get some conclusions
- print out locations with no EV charger station within 1 km
- print out top 10 locations with the most EV charger stations around

In [21]:
# print locations with no EV charger station within 1 km
li = []
n = 0
for location,number in landmarks_charger_number.items():
    if number == 0:
        li.append(location)
        n += 1
print(n,'landmarks in city of Melbourne has no EV charger station within 1 km:')
print('')
print('\n'.join(li))


23 landmarks in city of Melbourne has no EV charger station within 1 km:

Port of Melbourne
Carlton Football Club
Flemington Racecourse
Newmarket Reserve
Royal Park Railway Station
Flemington Bridge Railway Station
Royal Park Golf Course
Visy Park
Royal Park
Flemington Racecourse Railway Station
North Melbourne Recreation Centre (Aquatic)
North Melbourne Recreation Centre (Gymnasium)
Carlton Baths
State Netball Hockey Centre
Melbourne General Cemetery
Melbourne Zoo
Point Park
Richmond Railway Station (Richmond) - Train stop
North Melbourne Recreation Reserve
Princes Park
Showgrounds Railway Station (Flemington)
Newmarket Railway Station
Melbourne Showgrounds


In [20]:
# print 10 locations with the most EV charger stations within 1 km
print('10 locations with the most EV charger stations within 1 km in city of Melbourne:')
top10_landmarks = dict(sorted(landmarks_charger_number.items(), key=lambda item: item[1],reverse=True))

for name in list(top10_landmarks.keys())[:10]:
    print(name, landmarks_charger_number[name])

10 locations with the most EV charger stations within 1 km in city of Melbourne:
Melbourne Visitor Booth 44
Victoria Police 40
Melbourne Town Hall 40
Melbourne Childrens Court 36
Supreme Court 36
Melbourne Magistrates Court 35
Flinders Street Railway Station 35
County Court Melbourne 35
Sandridge Rail Bridge 35
Melbourne Visitor Centre 34


### <span style="color:red"><b>Visualization</b></span>

In [24]:
import geopandas
import folium

df = pd.read_csv('EV charger station info around public locations.csv')
print(df.head(10))

   Unnamed: 0                   Landmark Name   Latitude   Longitude  \
0           0               Port of Melbourne -37.813738  144.917534   
1           1           Carlton Football Club -37.784086  144.961968   
2           2           Carlton Gardens North -37.801769  144.971998   
3           3                    Kings Domain -37.825524  144.974108   
4           4           Flemington Racecourse -37.790808  144.912143   
5           5                Treasury Gardens -37.814399  144.975952   
6           6     Melbourne Magistrates Court -37.813615  144.956846   
7           7       Flagstaff Railway Station -37.811638  144.956119   
8           8  Melbourne Cricket Ground (MCG) -37.819492  144.983403   
9           9          Queen Victoria Gardens -37.821638  144.971050   

   Number of EV charger station  \
0                             0   
1                             0   
2                             6   
3                             9   
4                             0 

In [54]:
#import branca.colormap as cm

# Create a plain map centered around Melbourne
melbourne_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=12, tiles='cartodb positron')

# Add markers for each landmark
for index, row in df.iterrows():
    # Determine color based on the number of EV stations
    color = 'white'

    if row['Number of EV charger station'] > 40:
        color = 'darkred'
    elif row['Number of EV charger station'] > 30:
        color = 'red'
    elif row['Number of EV charger station'] > 20:
        color = 'orange'
    elif row['Number of EV charger station'] > 10:
        color = 'lightred'
    elif row['Number of EV charger station'] > 0:
        color =  'pink'
    else:
        color = 'gray'

    color_density = cm.linear.YlOrRd_04.to_step(10).scale(0, 44)
    
    #density = row['Number of EV charger station']
    #color = colormap(density)
    
    
    # Add a marker for each destination
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"<b>{row['Landmark Name']}</b><br>EV stations: {row['Number of EV charger station']}",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(melbourne_map)

    # Add circle with radius 1 km around each landmark
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        radius=1000,  # 1 km in meters
        color=None,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
    ).add_to(melbourne_map)


# Save the map as an HTML file
melbourne_map.save('melbourne_ev_stations_map_2.html')

# Display the map
melbourne_map

